In [97]:
import pandas as pd
ret = pd.read_csv('../data/mid_cap_all_sectors_ret.csv', index_col='date') * 100

In [98]:
ret = ret.stack()

In [99]:
df = ret.reset_index()
df.columns = ['date', 'sector', 'ret']
df = df.set_index(['date', 'sector'])

In [100]:
unique_dates = df.index.get_level_values('date').unique()

n = int(len(unique_dates) * 0.8)
train_n = int(n * 0.95)
tmp_dates = unique_dates[:n]
train_dates = tmp_dates[:train_n]
valid_dates = tmp_dates[train_n:]

train_df = df.loc[train_dates]
valid_df = df.loc[valid_dates]

In [104]:
train_df.loc[(slice(None), "Materials"), :]['ret']

date        sector   
2000-01-04  Materials   -1.115250
2000-01-05  Materials    1.450299
2000-01-06  Materials    1.915120
2000-01-07  Materials    0.146314
2000-01-10  Materials    0.235475
                           ...   
2018-09-21  Materials   -0.529251
2018-09-24  Materials   -0.808885
2018-09-25  Materials   -0.522333
2018-09-26  Materials   -0.771957
2018-09-27  Materials   -0.430742
Name: ret, Length: 4714, dtype: float64

In [94]:
valid_df

ret
date       sector                          
2018-09-28 Materials              -0.258594
           Industrials             0.208235
           Health Care             0.987667
           Real Estate             1.560265
           Consumer Discretionary -0.313496
...                                     ...
2019-09-25 Utilities               0.562569
           Information Technology  2.061007
           Energy                  0.365385
           Consumer Staples        0.863540
           Communication Services  1.320629

[2739 rows x 1 columns]

In [44]:
import torch
import torch.nn as nn
import numpy as np

def data_to_tensor(data, dtype=torch.float32):
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    return torch.tensor(np.array(data), dtype=dtype).to(device)

class CnnSeqDataset(torch.utils.data.Dataset):
    def __init__(self, data, selected_features, seq_n):
        df = data.swaplevel().sort_index().copy()
        sample_index = df.groupby('sector')[selected_features].shift(seq_n-1).dropna().index.tolist()
        inputs = df.loc[:, selected_features]

        self.data_list = []
        for sample in sample_index:
            data_tensor = data_to_tensor(inputs.loc[:sample].iloc[-seq_n:].T)
            data_tuple = (data_tensor, data_tensor)
            self.data_list.append(data_tuple)

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        return self.data_list[idx]


In [45]:
dataset = Dataset(df, ['ret'], 20)

[('Communication Services', '2000-02-01'), ('Communication Services', '2000-02-02'), ('Communication Services', '2000-02-03'), ('Communication Services', '2000-02-04'), ('Communication Services', '2000-02-07'), ('Communication Services', '2000-02-08'), ('Communication Services', '2000-02-09'), ('Communication Services', '2000-02-10'), ('Communication Services', '2000-02-11'), ('Communication Services', '2000-02-14'), ('Communication Services', '2000-02-15'), ('Communication Services', '2000-02-16'), ('Communication Services', '2000-02-17'), ('Communication Services', '2000-02-18'), ('Communication Services', '2000-02-22'), ('Communication Services', '2000-02-23'), ('Communication Services', '2000-02-24'), ('Communication Services', '2000-02-25'), ('Communication Services', '2000-02-28'), ('Communication Services', '2000-02-29')]


In [75]:
len(dataset)

68035